In [ ]:
# %% [markdown]
# # Load and Inspect Preprocessed RNA-seq Datasets
#
# This notebook loads the AnnData files generated by Stage 2.5 of the pipeline
# (`preprocess_dataset_gene_ids.py`) and performs basic checks and inspections.

# %%
import scanpy as sc
import pandas as pd
import os
import numpy as np # For isnan check

# Configure pandas display options for better viewing
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# %%
# --- Configuration ---
# Directory containing the preprocessed h5ad files
preprocessed_dir = "/mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/preprocessed_data/run_20250502_211754/"

# List of dataset prefixes to load
dataset_names = ['adni', 'encode', 'gtex', 'mage'] # Add 'entex' if it was processed

# --- End Configuration ---

# %%
# Dictionary to hold the loaded AnnData objects
adata_dict = {}

print(f"Loading datasets from: {preprocessed_dir}\n")

for name in dataset_names:
    file_name = f"{name}_standardized_preprocessed.h5ad"
    file_path = os.path.join(preprocessed_dir, file_name)
    print(f"--- Loading {name.upper()} ---")
    print(f"File: {file_path}")
    if os.path.exists(file_path):
        try:
            adata_dict[name] = sc.read_h5ad(file_path)
            print(f"Successfully loaded {name.upper()} with shape {adata_dict[name].shape}")
        except Exception as e:
            print(f"ERROR: Could not load {file_path}: {e}")
    else:
        print(f"WARNING: File not found: {file_path}")
    print("-" * (len(name) + 14))

# %% [markdown]
# ## Inspect Loaded Datasets

# %%
if not adata_dict:
    print("No datasets were loaded successfully. Cannot perform inspection.")
else:
    for name, adata in adata_dict.items():
        print(f"\n\n========================= Inspecting: {name.upper()} =========================\n")

        # --- Basic Info ---
        print(f"--- AnnData Object Info ({name.upper()}) ---")
        print(adata)
        print("-" * 30)

        # --- Observations (.obs) ---
        print(f"\n--- Observations (.obs) Info ({name.upper()}) ---")
        print(f".obs shape: {adata.obs.shape}")
        print(f".obs columns: {adata.obs.columns.tolist()}")
        print("\n.obs.head():")
        print(adata.obs.head())
        # Check unique values for key categorical fields
        print("\nUnique values in key .obs columns:")
        for col in ['dataset', 'tissue', 'sex', 'data_type', 'expression_unit', 'tissue_ontology']:
            if col in adata.obs.columns:
                try:
                    unique_vals = adata.obs[col].unique()
                    # Handle categorical output nicely
                    if isinstance(unique_vals, pd.Categorical):
                         unique_vals = unique_vals.tolist()
                    # Limit display if too many unique values (e.g., tissue in GTEx)
                    display_vals = unique_vals[:10] if len(unique_vals) > 10 else unique_vals
                    ellipsis = "..." if len(unique_vals) > 10 else ""
                    print(f"  '{col}': {display_vals}{ellipsis} ({len(unique_vals)} total unique)")
                except Exception as e:
                    print(f"  Could not get unique values for '{col}': {e}")
            else:
                print(f"  '{col}': Column not found")
        # Check for missing values
        print("\nMissing values count in key .obs columns:")
        missing_data = {}
        for col in ['tissue', 'subject_id', 'dataset', 'sex', 'age', 'tissue_ontology']:
             if col in adata.obs.columns:
                # Count NaNs directly
                na_count = adata.obs[col].isna().sum()
                # Count empty strings '', being careful about NaNs
                is_empty_series = adata.obs[col].apply(lambda x: isinstance(x, str) and x.strip() == '')
                try:
                    empty_count = np.sum(is_empty_series.to_numpy(dtype=bool))
                except: # Fallback if conversion fails
                    empty_count = sum(1 for x in is_empty_series if x is True)
                total_missing = na_count + empty_count
                if total_missing > 0:
                    missing_data[col] = total_missing
        if missing_data:
            print(missing_data)
        else:
             print("  No missing/empty values found in checked key columns.")
        print("-" * 30)


        # --- Variables (.var) ---
        print(f"\n--- Variables (.var) Info ({name.upper()}) ---")
        print(f".var shape: {adata.var.shape}")
        print(f".var columns: {adata.var.columns.tolist()}")
        print(f".var index name: {adata.var.index.name}")
        print(f"First 5 var_names (index): {adata.var_names[:5].tolist()}")
        print("\n.var.head():")
        print(adata.var.head())
         # Check unique values for key categorical fields
        print("\nUnique values in key .var columns:")
        for col in ['gene_type', 'chromosome', 'mapping_source', 'mapping_confidence']:
            if col in adata.var.columns:
                try:
                    unique_vals = adata.var[col].unique()
                     # Handle categorical output nicely
                    if isinstance(unique_vals, pd.Categorical):
                         unique_vals = unique_vals.tolist()
                    display_vals = unique_vals[:10] if len(unique_vals) > 10 else unique_vals
                    ellipsis = "..." if len(unique_vals) > 10 else ""
                    print(f"  '{col}': {display_vals}{ellipsis} ({len(unique_vals)} total unique)")
                except Exception as e:
                    print(f"  Could not get unique values for '{col}': {e}")
            else:
                 print(f"  '{col}': Column not found")
        print("-" * 30)

        # --- Unstructured Annotation (.uns) ---
        print(f"\n--- Unstructured Annotation (.uns) Info ({name.upper()}) ---")
        print(f".uns keys: {list(adata.uns.keys())}")
        # Print specific .uns fields if they exist
        for key in ['dataset_info', 'gene_mapping_stats', 'metadata_validation', 'harmonized_gencode_version', 'harmonized_reference_genome']:
            if key in adata.uns:
                print(f"\n.uns['{key}']: ")
                # Pretty print dictionaries
                if isinstance(adata.uns[key], dict):
                    import json
                    print(json.dumps(adata.uns[key], indent=2, default=str)) # Use default=str for non-serializable
                else:
                    print(adata.uns[key])
        print("-" * 30)

print("\n========================= Inspection Complete =========================")

Loading datasets from: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/preprocessed_data/run_20250502_211754/

--- Loading ADNI ---
File: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/preprocessed_data/run_20250502_211754/adni_standardized_preprocessed.h5ad
Successfully loaded ADNI with shape (650, 17991)
------------------
--- Loading ENCODE ---
File: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/preprocessed_data/run_20250502_211754/encode_standardized_preprocessed.h5ad
Successfully loaded ENCODE with shape (7, 65586)
--------------------
--- Loading GTEX ---
File: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/preprocessed_data/run_20250502_211754/gtex_standardized_preprocessed.h5ad
Successfully loaded GTEX with shape (19616, 58988)
------------------
--- Loading MAGE ---
File: /mnt/czi-sci-ai/intrinsic-variation-gene-ex/rnaseq/preprocessed_data/run_20250502_211754/mage_standardized_preprocessed.h5ad
Successfully loaded MAGE with shape (731, 19428)
-----------------